# 按照条件过滤经过相似性筛选的点位

In [1]:
import arcpy
import os
import pandas as pd


In [24]:
# 读取文件
df_filter = pd.read_csv(r"F:\cache_data\frequency_filter\dy\scaler_csv\dy_stander_filter_all_type_20240417.csv")

## 1、标记在非耕地上的点位
### 得到非耕地字典

In [23]:
# 获取DL代码对照表
dl_polygon_path = r'D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features_dy\dy_sd'
df_dl_polygon=pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(dl_polygon_path,['DLMC','BM']))
# 得到每个DLDM的DLMC
dl_dict = df_dl_polygon.set_index('BM')['DLMC'].to_dict()
dl_dict

{1: '水田',
 2: '水浇地',
 3: '旱地',
 4: '果园',
 5: '茶园',
 6: '其他园地',
 7: '乔木林地',
 8: '竹林地',
 9: '灌木林地',
 10: '其他林地',
 11: '人工牧草地',
 12: '其他草地',
 13: '物流仓储用地',
 14: '商业服务业设施用地',
 15: '工业用地',
 16: '采矿用地',
 17: '城镇住宅用地',
 18: '设施农用地',
 19: '农村宅基地',
 20: '裸岩石砾地',
 21: '公用设施用地',
 22: '公园与绿地',
 23: '广场用地',
 24: '机关团体新闻出版用地',
 25: '科教文卫用地',
 26: '高教用地',
 27: '特殊用地',
 28: '交通服务场站用地',
 29: '管道运输用地',
 30: '水库水面',
 31: '坑塘水面',
 32: '养殖坑塘',
 33: '内陆滩涂',
 34: '水工建筑用地',
 35: '空闲地',
 36: '裸土地',
 37: '农村道路',
 38: '公路用地',
 39: '河流水面',
 40: '铁路用地',
 41: '城镇村道路用地',
 42: '沟渠'}

In [25]:
# 非耕地代码
fgd_code = [13,14,17,19,21,23,24,25,26,27,28,38,40,41]
df_filter['FGD_LABEL'] = df_filter['DL'].apply(lambda x: 1 if x in fgd_code else 0) # 1为非耕地，0为耕地

### 2、查询水稻土点位是否位于水田

In [26]:
# 水田代码
st_code = [1,2,3,30,31,32,39]
df_filter['ST_CODE'] =  ((df_filter['TL']=='水稻土') & (df_filter['DL'].isin(st_code))).astype(int) # 1:符合水稻土，0:不符合水稻土

### 3、自然土类位于自然土类代码中

In [27]:
# 自然土代码
zrt_code = [4,5,6,7,8,9,10,11,12,20,35,36]
# 自然土土壤类型
zrt_tl_str = '薄腐'
# df_filter['ZRT_LABEL'] = ((df_filter['TZ'].str.contains(zrt_tl_str)) & (df_filter['DL'].isin(zrt_code))).astype(int) # 1:符合自然土，0:不符合自然土
df_filter['ZRT_LABEL'] = df_filter['DL'].apply(lambda x: 1 if x in zrt_code else 0) # 1为自然土，0为非自然土

### 4、图层厚度获取

In [28]:
# 薄层
bc_value = 0.5
# 中层
mc_value = 1

def hd_filter(value):
    if value <= bc_value:
        return 0
    elif value <= mc_value:
        return 1
    else:
        return 2
df_filter['SOIL_HD'] = df_filter['MRVBF'].apply(hd_filter)

In [29]:
df_filter.to_csv(r"C:\Users\Runker\Desktop\test.csv",index=False)

In [47]:
def calc(x):
    if "厚腐" in x:
        return x.replace("厚腐","中腐")
    elif "中腐" in x:
        return x.replace("中腐","薄腐")
    else:
        return x


In [48]:
s = '中腐殖层厚土层硅铁质黄壤'

In [45]:
result = s.replace("中腐","薄腐")

In [46]:
result

'薄腐殖层厚土层硅铁质黄壤'

In [44]:
s

'中腐殖层厚土层硅铁质黄壤'

In [49]:
calc(s)

'薄腐殖层厚土层硅铁质黄壤'

In [50]:
a_dict = {0: ['中白鳝泥田'], 1: ['中腐殖层中土层硅质黄壤'], 2: ['中腐殖层中土层黄色石灰土'], 3: ['中腐殖层厚土层硅质黄壤'], 4: ['中腐殖层薄土层硅质黄壤'], 5: ['中腐殖层薄土层黄色石灰土'], 6: ['偏沙土'], 7: ['冷水田'], 8: ['冷沙土'], 9: ['冷沙田'], 10: ['冷浸田'], 11: ['大泥土'], 12: ['大泥田'], 13: ['大眼泥田'], 14: ['大眼黄泥田'], 15: ['小泥土'], 16: ['小黄泥土'], 17: ['小黄泥田'], 18: ['岩泥'], 19: ['干鸭屎泥田'], 20: ['扁沙泥田'], 21: ['死黄泥土'], 22: ['死黄泥田'], 23: ['浅足烂泥田'], 24: ['深足烂泥田'], 25: ['湿鸭屎泥田'], 26: ['潮板沙田'], 27: ['潮沙泥田'], 28: ['潮沙田'], 29: ['潮油沙泥田'], 30: ['潮泥田'], 31: ['火石沙土'], 32: ['烂锈田'], 33: ['熟鸭屎泥田'], 34: ['白云沙土'], 35: ['白沙土'], 36: ['白沙泥田'], 37: ['石碴子土'], 38: ['砾质紫沙土'], 39: ['紫沙土'], 40: ['紫色土'], 41: ['苦大泥田'], 42: ['薄腐殖中土层山地灌丛草甸土'], 43: ['薄腐殖层中土层山地灌丛草甸土'], 44: ['薄腐殖层中土层次生黄色石灰土'], 45: ['薄腐殖层中土层硅质黄壤'], 46: ['薄腐殖层中土层硅铁质黄壤'], 47: ['薄腐殖层中土层硅铝质黄壤'], 48: ['薄腐殖层中土层紫色土'], 49: ['薄腐殖层中土层铁铝质黄壤'], 50: ['薄腐殖层中土层黄色石灰土'], 51: ['薄腐殖层中土层黑色石灰土'], 52: ['薄腐殖层厚土层山地灌丛草甸土'], 53: ['薄腐殖层厚土层次生黄色石灰土'], 54: ['薄腐殖层厚土层硅质黄壤'], 55: ['薄腐殖层厚土层硅铁质黄壤'], 56: ['薄腐殖层厚土层硅铝质黄壤'], 57: ['薄腐殖层厚土层铁铝质黄壤'], 58: ['薄腐殖层厚土层黄色石灰土'], 59: ['薄腐殖层厚土层黑色石灰土'], 60: ['薄腐殖层薄土层山地灌丛草甸土'], 61: ['薄腐殖层薄土层次生黄色石灰土'], 62: ['薄腐殖层薄土层淋溶黄色石灰土'], 63: ['薄腐殖层薄土层硅质黄壤'], 64: ['薄腐殖层薄土层硅铁质黄壤'], 65: ['薄腐殖层薄土层硅铝质黄壤'], 66: ['薄腐殖层薄土层紫色土'], 67: ['薄腐殖层薄土层酸性紫色土'], 68: ['薄腐殖层薄土层铁质黄壤'], 69: ['薄腐殖层薄土层铁铝质黄壤'], 70: ['薄腐殖层薄土层黄壤性土'], 71: ['薄腐殖层薄土层黄色石灰土'], 72: ['薄腐殖层薄土层黑色石灰土'], 73: ['薄腐殖薄土层山地灌丛草甸土'], 74: ['重白胶泥田'], 75: ['重白鳝泥田'], 76: ['钙石子土'], 77: ['铅毒田'], 78: ['锈水田'], 79: ['雾黄泥田'], 80: ['马血泥田'], 81: ['黄大泥田'], 82: ['黄沙泥土'], 83: ['黄沙泥田'], 84: ['黄油沙泥田'], 85: ['黄泥土'], 86: ['黄泥田'], 87: ['黄石砂田'], 88: ['黄胶泥土'], 89: ['黄胶泥田'], 90: ['黑潮泥田'], 91: ['龙凤大眼泥田']}
b_dict = {0: ['中白鳝泥田'], 1: ['偏沙土'], 2: ['冷水田'], 3: ['冷沙土'], 4: ['冷沙田'], 5: ['冷浸田'], 6: ['厚腐殖层中土层硅质黄壤'], 7: ['厚腐殖层中土层黄色石灰土'], 8: ['厚腐殖层薄土层硅质黄壤'], 9: ['大泥土'], 10: ['大泥田'], 11: ['大眼泥田'], 12: ['大眼黄泥田'], 13: ['小泥土'], 14: ['小黄泥土'], 15: ['小黄泥田'], 16: ['岩泥'], 17: ['干鸭屎泥田'], 18: ['扁沙泥田'], 19: ['死黄泥土'], 20: ['死黄泥田'], 21: ['浅足烂泥田'], 22: ['深足烂泥田'], 23: ['湿鸭屎泥田'], 24: ['潮板沙田'], 25: ['潮沙泥田'], 26: ['潮沙田'], 27: ['潮油沙泥田'], 28: ['潮泥田'], 29: ['火石沙土'], 30: ['烂锈田'], 31: ['熟鸭屎泥田'], 32: ['白云沙土'], 33: ['白沙土'], 34: ['白沙泥田'], 35: ['石碴子土'], 36: ['砾质紫沙土'], 37: ['紫沙土'], 38: ['紫色土'], 39: ['苦大泥田'], 40: ['薄腐殖层中土层山地灌丛草甸土'], 41: ['薄腐殖层中土层次生黄色石灰土'], 42: ['薄腐殖层中土层硅铁质黄壤'], 43: ['薄腐殖层中土层硅铝质黄壤'], 44: ['薄腐殖层中土层紫色土'], 45: ['薄腐殖层中土层酸性紫色土'], 46: ['薄腐殖层中土层铁铝质黄壤'], 47: ['薄腐殖层中土层黄壤性土'], 48: ['薄腐殖层中土层黄色石灰土'], 49: ['薄腐殖层中土层黑色石灰土'], 50: ['薄腐殖层厚土层次生黄色石灰土'], 51: ['薄腐殖层厚土层淋溶黄色石灰土'], 52: ['薄腐殖层厚土层硅质黄壤'], 53: ['薄腐殖层厚土层硅铁质黄壤'], 54: ['薄腐殖层厚土层硅铝质黄壤'], 55: ['薄腐殖层厚土层紫色土'], 56: ['薄腐殖层厚土层铁铝质黄壤'], 57: ['薄腐殖层厚土层黄色石灰土'], 58: ['薄腐殖层厚土层黑色石灰土'], 59: ['薄腐殖层薄土层硅质黄壤'], 60: ['薄腐殖层薄土层硅铁质黄壤'], 61: ['薄腐殖层薄土层硅铝质黄壤'], 62: ['薄腐殖层薄土层铁质黄壤'], 63: ['薄腐殖层薄土层铁铝质黄壤'], 64: ['薄腐殖层薄土层黄色石灰土'], 65: ['薄腐殖层薄土层黑色石灰土'], 66: ['薄腐殖薄土层山地灌丛草甸土'], 67: ['重白胶泥田'], 68: ['重白鳝泥田'], 69: ['钙石子土'], 70: ['铅毒田'], 71: ['锈水田'], 72: ['雾黄泥田'], 73: ['马血泥田'], 74: ['黄大泥田'], 75: ['黄沙泥土'], 76: ['黄沙泥田'], 77: ['黄油沙泥田'], 78: ['黄泥土'], 79: ['黄泥田'], 80: ['黄石砂田'], 81: ['黄胶泥土'], 82: ['黄胶泥田'], 83: ['黑潮泥田'], 84: ['龙凤大眼泥田']}

In [58]:
a_set = set([_[0] for _ in a_dict.values()])
b_set = set([_[0] for _ in b_dict.values()])


In [60]:
all = a_set | b_set

In [63]:
len(a_set),len(b_set)

(92, 85)

In [64]:
all,len(all)

({'中白鳝泥田',
  '中腐殖层中土层硅质黄壤',
  '中腐殖层中土层黄色石灰土',
  '中腐殖层厚土层硅质黄壤',
  '中腐殖层薄土层硅质黄壤',
  '中腐殖层薄土层黄色石灰土',
  '偏沙土',
  '冷水田',
  '冷沙土',
  '冷沙田',
  '冷浸田',
  '厚腐殖层中土层硅质黄壤',
  '厚腐殖层中土层黄色石灰土',
  '厚腐殖层薄土层硅质黄壤',
  '大泥土',
  '大泥田',
  '大眼泥田',
  '大眼黄泥田',
  '小泥土',
  '小黄泥土',
  '小黄泥田',
  '岩泥',
  '干鸭屎泥田',
  '扁沙泥田',
  '死黄泥土',
  '死黄泥田',
  '浅足烂泥田',
  '深足烂泥田',
  '湿鸭屎泥田',
  '潮板沙田',
  '潮沙泥田',
  '潮沙田',
  '潮油沙泥田',
  '潮泥田',
  '火石沙土',
  '烂锈田',
  '熟鸭屎泥田',
  '白云沙土',
  '白沙土',
  '白沙泥田',
  '石碴子土',
  '砾质紫沙土',
  '紫沙土',
  '紫色土',
  '苦大泥田',
  '薄腐殖中土层山地灌丛草甸土',
  '薄腐殖层中土层山地灌丛草甸土',
  '薄腐殖层中土层次生黄色石灰土',
  '薄腐殖层中土层硅质黄壤',
  '薄腐殖层中土层硅铁质黄壤',
  '薄腐殖层中土层硅铝质黄壤',
  '薄腐殖层中土层紫色土',
  '薄腐殖层中土层酸性紫色土',
  '薄腐殖层中土层铁铝质黄壤',
  '薄腐殖层中土层黄壤性土',
  '薄腐殖层中土层黄色石灰土',
  '薄腐殖层中土层黑色石灰土',
  '薄腐殖层厚土层山地灌丛草甸土',
  '薄腐殖层厚土层次生黄色石灰土',
  '薄腐殖层厚土层淋溶黄色石灰土',
  '薄腐殖层厚土层硅质黄壤',
  '薄腐殖层厚土层硅铁质黄壤',
  '薄腐殖层厚土层硅铝质黄壤',
  '薄腐殖层厚土层紫色土',
  '薄腐殖层厚土层铁铝质黄壤',
  '薄腐殖层厚土层黄色石灰土',
  '薄腐殖层厚土层黑色石灰土',
  '薄腐殖层薄土层山地灌丛草甸土',
  '薄腐殖层薄土层次生黄色石灰土',
  '薄腐殖层薄土层淋溶黄色石灰土',
  '薄腐殖层薄土层硅质黄壤',
  '薄腐殖层薄土层硅铁质黄壤',
  '

In [65]:
union_value = a_set & b_set
union_value,len(union_value)

({'中白鳝泥田',
  '偏沙土',
  '冷水田',
  '冷沙土',
  '冷沙田',
  '冷浸田',
  '大泥土',
  '大泥田',
  '大眼泥田',
  '大眼黄泥田',
  '小泥土',
  '小黄泥土',
  '小黄泥田',
  '岩泥',
  '干鸭屎泥田',
  '扁沙泥田',
  '死黄泥土',
  '死黄泥田',
  '浅足烂泥田',
  '深足烂泥田',
  '湿鸭屎泥田',
  '潮板沙田',
  '潮沙泥田',
  '潮沙田',
  '潮油沙泥田',
  '潮泥田',
  '火石沙土',
  '烂锈田',
  '熟鸭屎泥田',
  '白云沙土',
  '白沙土',
  '白沙泥田',
  '石碴子土',
  '砾质紫沙土',
  '紫沙土',
  '紫色土',
  '苦大泥田',
  '薄腐殖层中土层山地灌丛草甸土',
  '薄腐殖层中土层次生黄色石灰土',
  '薄腐殖层中土层硅铁质黄壤',
  '薄腐殖层中土层硅铝质黄壤',
  '薄腐殖层中土层紫色土',
  '薄腐殖层中土层铁铝质黄壤',
  '薄腐殖层中土层黄色石灰土',
  '薄腐殖层中土层黑色石灰土',
  '薄腐殖层厚土层次生黄色石灰土',
  '薄腐殖层厚土层硅质黄壤',
  '薄腐殖层厚土层硅铁质黄壤',
  '薄腐殖层厚土层硅铝质黄壤',
  '薄腐殖层厚土层铁铝质黄壤',
  '薄腐殖层厚土层黄色石灰土',
  '薄腐殖层厚土层黑色石灰土',
  '薄腐殖层薄土层硅质黄壤',
  '薄腐殖层薄土层硅铁质黄壤',
  '薄腐殖层薄土层硅铝质黄壤',
  '薄腐殖层薄土层铁质黄壤',
  '薄腐殖层薄土层铁铝质黄壤',
  '薄腐殖层薄土层黄色石灰土',
  '薄腐殖层薄土层黑色石灰土',
  '薄腐殖薄土层山地灌丛草甸土',
  '重白胶泥田',
  '重白鳝泥田',
  '钙石子土',
  '铅毒田',
  '锈水田',
  '雾黄泥田',
  '马血泥田',
  '黄大泥田',
  '黄沙泥土',
  '黄沙泥田',
  '黄油沙泥田',
  '黄泥土',
  '黄泥田',
  '黄石砂田',
  '黄胶泥土',
  '黄胶泥田',
  '黑潮泥田',
  '龙凤大眼泥田'},
 78)

In [ ]:
({'中白鳝泥田',
  '偏沙土',
  '冷水田',
  '冷沙土',
  '冷沙田',
  '冷浸田',
  '大泥土',
  '大泥田',
  '大眼泥田',
  '大眼黄泥田',
  '小泥土',
  '小黄泥土',
  '小黄泥田',
  '岩泥',
  '干鸭屎泥田',
  '扁沙泥田',
  '死黄泥土',
  '死黄泥田',
  '浅足烂泥田',
  '深足烂泥田',
  '湿鸭屎泥田',
  '潮板沙田',
  '潮沙泥田',
  '潮沙田',
  '潮油沙泥田',
  '潮泥田',
  '火石沙土',
  '烂锈田',
  '熟鸭屎泥田',
  '白云沙土',
  '白沙土',
  '白沙泥田',
  '石碴子土',
  '砾质紫沙土',
  '紫沙土',
  '紫色土',
  '苦大泥田',
  '薄腐殖层中土层山地灌丛草甸土',
  '薄腐殖层中土层次生黄色石灰土',
  '薄腐殖层中土层硅铁质黄壤',
  '薄腐殖层中土层硅铝质黄壤',
  '薄腐殖层中土层紫色土',
  '薄腐殖层中土层铁铝质黄壤',
  '薄腐殖层中土层黄色石灰土',
  '薄腐殖层中土层黑色石灰土',
  '薄腐殖层厚土层次生黄色石灰土',
  '薄腐殖层厚土层硅质黄壤',
  '薄腐殖层厚土层硅铁质黄壤',
  '薄腐殖层厚土层硅铝质黄壤',
  '薄腐殖层厚土层铁铝质黄壤',
  '薄腐殖层厚土层黄色石灰土',
  '薄腐殖层厚土层黑色石灰土',
  '薄腐殖层薄土层硅质黄壤',
  '薄腐殖层薄土层硅铁质黄壤',
  '薄腐殖层薄土层硅铝质黄壤',
  '薄腐殖层薄土层铁质黄壤',
  '薄腐殖层薄土层铁铝质黄壤',
  '薄腐殖层薄土层黄色石灰土',
  '薄腐殖层薄土层黑色石灰土',
  '薄腐殖薄土层山地灌丛草甸土',
  '重白胶泥田',
  '重白鳝泥田',
  '钙石子土',
  '铅毒田',
  '锈水田',
  '雾黄泥田',
  '马血泥田',
  '黄大泥田',
  '黄沙泥土',
  '黄沙泥田',
  '黄油沙泥田',
  '黄泥土',
  '黄泥田',
  '黄石砂田',
  '黄胶泥土',
  '黄胶泥田',
  '黑潮泥田',
  '龙凤大眼泥田'},
 78)

In [66]:
a_alone = a_set-union_value
a_alone,len(a_alone)

({'中腐殖层中土层硅质黄壤',
  '中腐殖层中土层黄色石灰土',
  '中腐殖层厚土层硅质黄壤',
  '中腐殖层薄土层硅质黄壤',
  '中腐殖层薄土层黄色石灰土',
  '薄腐殖中土层山地灌丛草甸土',
  '薄腐殖层中土层硅质黄壤',
  '薄腐殖层厚土层山地灌丛草甸土',
  '薄腐殖层薄土层山地灌丛草甸土',
  '薄腐殖层薄土层次生黄色石灰土',
  '薄腐殖层薄土层淋溶黄色石灰土',
  '薄腐殖层薄土层紫色土',
  '薄腐殖层薄土层酸性紫色土',
  '薄腐殖层薄土层黄壤性土'},
 14)

In [67]:
b_alone = b_set-union_value
b_alone,len(b_alone)

({'厚腐殖层中土层硅质黄壤',
  '厚腐殖层中土层黄色石灰土',
  '厚腐殖层薄土层硅质黄壤',
  '薄腐殖层中土层酸性紫色土',
  '薄腐殖层中土层黄壤性土',
  '薄腐殖层厚土层淋溶黄色石灰土',
  '薄腐殖层厚土层紫色土'},
 7)